# Sequence Hacking, Hashing, and Slicing

In [ ]:
from array import array
import functools
import math
import numbers
import operator
import reprlib
import itertools

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other: 'Vector'):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self._components))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
        
    def angle(self, n):
        r = math.sqrt(sum(x*x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi *2 -a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<[]>'
        else:
            coords = self
            outer_fmt = '([])'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))


    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)


v1 = Vector([3,4,5])
print(v1[0] , v1[-1])
v7 = Vector(range(7))
print(v7[1:4])
print(v7)
print(v7[1])
print(v7[-1:])
v7[1, 2]


    


3.0 5.0
(1.0, 2.0, 3.0)
(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)
1.0
(6.0,)


TypeError: Vector indices must be integers

## Protocols and Duck Typing
Protocols are informal interfaces that are defined in documentation but not in code. <br>
For example the sequence protocol in Python entails just the `__len__` and `__getitem__` methods. <br>
Any class that implements these two methods can be used where a sequence is expected in python, regardless of the class parents and inheritance. <br>
So we usually say that an object is a sequence just because it behaves like one, this what duck typing is about. <br>
Because the protocol is informal, it could be implemented just half way.

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11) + list('JQKA')]
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for rank in self.ranks for suit in self.suits]

    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, index):
        return self._cards[index]
    

### How slicing works

In [3]:
class MySeq:
    def __getitem__(self, index):
        return index
    
s = MySeq()
print(s[1])
print(s[1:4])
print(s[1:4:2])
print(s[1:4:2, 9])
print(s[1:4:2, 7:9])

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


Let us take a closer look at slice

In [5]:
# Inspecting the attributes of the slice class

print(slice)
print(dir(slice))
print(help(slice.indices))

<class 'slice'>
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']
Help on method_descriptor:

indices(self, object, /) unbound builtins.slice method
    S.indices(len) -> (start, stop, stride)

    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.

None


In [6]:
# Examples

print(slice(None, 10, 2).indices(5))
print(slice(-3, None, None).indices(5))

(0, 5, 2)
(2, 5, 1)


### Dynamic attribute access
The `__getattr__` dunder is called when an attribute lookup fails

## Hashing
In the case of an object with many attributes, to apply the xor operator `functools.reduce` can be really useful

In [1]:
# Use reduce to compute 5!

import functools

functools.reduce(lambda a,b: a*b, range(1,6))

120

In [2]:
# Three ways to compute the aggregate xor

#1
n = 0
for i in range(1,6):
    n^=i

#2
import functools
print(functools.reduce(lambda a, b: a^b, range(6)))

#3
import operator
print(functools.reduce(operator.xor, range(6)))

1
1


## Map reduce computation

## Formatting